Documentação
https://docs.microsoft.com/en-us/python/api/overview/azure/storage-blob-readme?view=azure-python
https://docs.microsoft.com/en-us/python/api/azure-storage-blob/azure.storage.blob.blobclient?view=azure-python


Importação dos pacotes

In [66]:
import pyodbc as PDB  # Drive para conexão com banco de dados
import pandas  as pd
import sys
import os
import traceback
import json

Pacotes do Azure storage


In [177]:
pip install azure-storage-blob


SyntaxError: invalid syntax (<ipython-input-177-d025e5e1d416>, line 2)

In [67]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

In [68]:
print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")

Azure Blob Storage v12.8.0 - Python quickstart sample


Conexão com o sql server


In [69]:

config_file = open(r"D:\Ondrive\OneDrive - Implanta Informatica\credenciais.json")

config_storage = open(r"D:\Ondrive\OneDrive - Implanta Informatica\storage.json")

config_sql = json.load(config_file)

config_storage = json.load(config_storage)



In [78]:
server = config_sql["Servidores"][0]["server"]
database = '15-implanta'
username = config_sql["Servidores"][0]["username"]
password = config_sql["Servidores"][0]["password"]
driver= '{ODBC Driver 17 for SQL Server}'




Metodos auxiliares

In [79]:
def get_conectionOnDatabase(driver,server,database,username,password):
    _conexao = PDB.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
    return _conexao

In [80]:
conexao =get_conectionOnDatabase(driver,server,database,username,password)

In [ ]:
Executa a query para recuperar os dados 

In [81]:
SQL_Query = pd.read_sql_query(
'''SELECT  Top 10
       AA.IdArquivoAnexo,
       AA.IdEntidade,
       AA.Entidade,
       AA.Nome,
       AA.ContentType,
       AA.UrlStorageExterno,
       AA.NomeIdentificadorStorageExterno
  FROM Sistema.ArquivosAnexos AS AA
 
''', conexao)

conexao.close()

Monta o dataframe

In [83]:
registros = pd.DataFrame(SQL_Query, columns=['IdArquivoAnexo','IdEntidade','Entidade','Nome','ContentType','UrlStorageExterno','NomeIdentificadorStorageExterno'])

registros.set_index('IdArquivoAnexo',inplace=True)

Visualiza uma amostra

In [84]:
registros.sample(5)

ValueError: a must be greater than 0 unless no samples are taken

conecta no azure storage acouunt

In [85]:

connect_str_to_storage =config_storage["Storages"][0]["acesso"]
accountKey =config_storage["Storages"][0]["AccountKey"]


local_path =r'D:/Wesley/ArquivosAnexos'

In [86]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str_to_storage)

container_client = blob_service_client.get_container_client("oab-ba")

Executa o foreach para recuperar cada arquivo

In [ ]:
blob_list = container_client.list_blobs()

In [187]:
for index, row in registros.iterrows():
    nome_arquivo = row["UrlStorageExterno"]
    
    # Instantiate a new BlobClient
    # blob_client = container_client.get_blob_client(nome_arquivo)
    from azure.storage.blob import BlobClient
    blob_client = BlobClient.from_blob_url(blob_url=nome_arquivo,credential =accountKey)
    
    if(blob_client.exists()):
        file_name  =blob_client.blob_name

        file_name_origin = file_name[(file_name.rfind("/")+1):]
    
    
        target_diretorio = local_path +'/'+(str(index)+'='+file_name_origin)


        with open(target_diretorio, "wb") as my_blob:
            download_stream = blob_client.download_blob()
            my_blob.write(download_stream.readall())



#
